# Oncology Population Analysis

**Schemas:**
- `dim_patient` - Patient dimension with cohort attributes and cancer codes
- `dim_cancer_type` - Cancer type reference with ICD-10 ranges
- `dim_care_setting` - Care setting reference with detail (e.g., inpatient/acute inpatient, outpatient/emergency department)
- `dim_spend_bucket` - Spend tier segmentation
- `fact_oncology_cost` - Fact table with foreign keys to dimensions

In [ ]:
import duckdb

con = duckdb.connect('../local.duckdb', read_only=True)

con.sql("""
    SELECT name 
    FROM sqlite_master 
    WHERE type='table' 
    AND (name LIKE 'dim_%' OR name LIKE 'fact_oncology%')
    ORDER BY name
""").show()

┌──────────────────────────────┐
│             name             │
│           varchar            │
├──────────────────────────────┤
│ dim_cancer_type              │
│ dim_care_setting             │
│ dim_patient                  │
│ dim_spend_bucket             │
│ fact_oncology_cost           │
│ fact_oncology_cost_analytics │
│ fact_oncology_patient_detail │
└──────────────────────────────┘



## Dimension Tables Overview

#### Dim Cancer Type

In [87]:
con.sql("""
    SELECT 
        cancer_type_key,
        cancer_type_name,
        category,
        icd10_range_start,
        icd10_range_end
    FROM main.dim_cancer_type
    ORDER BY sort_order
""").df()

,cancer_type_key,cancer_type_name,category,icd10_range_start,icd10_range_end
0,1,"Lip, Oral Cavity & Pharynx",Malignant,C00,C15
1,2,Digestive Organs,Malignant,C15,C27
2,3,Respiratory & Intrathoracic,Malignant,C30,C40
3,4,Bone & Articular Cartilage,Malignant,C40,C42
4,5,Melanoma & Skin,Malignant,C43,C45
5,6,Mesothelial & Soft Tissue,Malignant,C45,C50
6,7,Breast,Malignant,C50,C51
7,8,Female Genital Organs,Malignant,C51,C59
8,9,Male Genital Organs,Malignant,C60,C64
9,10,Urinary Tract,Malignant,C64,C69


#### Dim Care Setting

In [88]:
con.sql("""
    SELECT 
        care_setting_key,
        care_setting_code,
        care_setting_name,
        care_setting_detail_code,
        care_setting_detail_name,
        sort_order
    FROM main.dim_care_setting
    ORDER BY sort_order, care_setting_detail_name
""").df()

,care_setting_key,care_setting_code,care_setting_name,care_setting_detail_code,care_setting_detail_name,sort_order
0,1,inpatient,Inpatient,acute inpatient,acute inpatient,1
1,2,inpatient,Inpatient,inpatient psychiatric,inpatient psychiatric,1
2,3,inpatient,Inpatient,inpatient rehabilitation,inpatient rehabilitation,1
3,4,inpatient,Inpatient,pharmacy,pharmacy,1
4,5,inpatient,Inpatient,skilled nursing,skilled nursing,1
5,6,outpatient,Outpatient,ambulatory surgery center,ambulatory surgery center,2
6,7,outpatient,Outpatient,dialysis,dialysis,2
7,8,outpatient,Outpatient,emergency department,emergency department,2
8,9,outpatient,Outpatient,home health,home health,2
9,10,outpatient,Outpatient,observation,observation,2


#### Dim Spend Bucket

In [89]:
con.sql("""
    SELECT *
    FROM main.dim_spend_bucket
    ORDER BY sort_order
""").df()

,spend_bucket_key,spend_bucket_name,min_threshold,max_threshold,sort_order
0,1,High Cost (>$100k),100000.0,NaN,1
1,2,Medium Cost ($25k-$100k),25000.0,99999.99,2
2,3,Low Cost ($5k-$25k),5000.0,24999.99,3
3,4,Minimal Cost (<$5k),0.0,4999.99,4


#### Dim Patient

In [90]:
cohort_df = con.sql("""
    SELECT 
        p.primary_cancer_type,
        ct.category,
        COUNT(*) as patient_count,
        AVG(p.cancer_claim_count) as avg_claims_per_patient
    FROM main.dim_patient p
    LEFT JOIN main.dim_cancer_type ct
        ON p.primary_cancer_type = ct.cancer_type_name
    GROUP BY 1, 2
    ORDER BY patient_count DESC
""").df()

cohort_df

,primary_cancer_type,category,patient_count,avg_claims_per_patient
0,Benign Neoplasms,Benign,241,7.489627
1,Uncertain Behavior,Uncertain,65,6.323077
2,Melanoma & Skin,Malignant,45,9.622222
3,Male Genital Organs,Malignant,33,17.060606
4,Other Neoplasm,Other,20,12.500000
5,Breast,Malignant,13,20.230769
6,Hematologic,Malignant,12,30.083333
7,Urinary Tract,Malignant,11,19.000000
8,Thyroid & Endocrine Glands,Malignant,6,21.333333
9,Digestive Organs,Malignant,6,5.500000


## Cost by Care Setting

In [91]:
cost_by_setting_df = con.sql("""
    SELECT 
        cs.care_setting_name,
        cs.care_setting_detail_name,
        SUM(f.paid_amount) as total_paid,
        SUM(f.claim_count) as total_claims,
        COUNT(DISTINCT f.person_id) as patients,
        ROUND(SUM(f.paid_amount) / NULLIF(COUNT(DISTINCT f.person_id), 0), 2) as avg_paid_per_patient,
        ROUND(SUM(f.paid_amount) / NULLIF(SUM(f.claim_count), 0), 2) as avg_paid_per_claim
    FROM main.fact_oncology_cost f
    JOIN main.dim_care_setting cs ON f.care_setting_key = cs.care_setting_key
    GROUP BY 1, 2
    ORDER BY care_setting_name, total_paid DESC
""").df()

cost_by_setting_df

,care_setting_name,care_setting_detail_name,total_paid,total_claims,patients,avg_paid_per_patient,avg_paid_per_claim
0,Ancillary,lab,8.533683e+05,17549.0,474,1800.36,48.63
1,Ancillary,ambulance,1.886267e+05,857.0,202,933.80,220.10
2,Ancillary,durable medical equipment,1.803833e+05,1332.0,286,630.71,135.42
3,Inpatient,acute inpatient,2.536291e+06,4151.0,240,10567.88,611.01
4,Inpatient,pharmacy,2.835269e+05,166.0,124,2286.51,1707.99
5,Inpatient,skilled nursing,1.134991e+05,313.0,41,2768.27,362.62
6,Inpatient,inpatient rehabilitation,2.958890e+03,35.0,6,493.15,84.54
7,Inpatient,inpatient psychiatric,2.257700e+02,5.0,5,45.15,45.15
8,Office Based,office based visit,8.377722e+05,11865.0,471,1778.71,70.61
9,Office Based,office based other,7.250250e+05,11564.0,469,1545.90,62.70


## Spend Distribution by Cancer Type

In [92]:
spend_dist_df = con.sql("""
    SELECT 
        ct.cancer_type_name,
        ct.category,
        sb.spend_bucket_name,
        COUNT(DISTINCT f.person_id) as patient_count,
        SUM(f.paid_amount) as total_paid,
        SUM(f.claim_count) as total_claims
    FROM main.fact_oncology_cost f
    JOIN main.dim_cancer_type ct ON f.cancer_type_key = ct.cancer_type_key
    JOIN main.dim_spend_bucket sb ON f.spend_bucket_key = sb.spend_bucket_key
    GROUP BY 1, 2, 3
    ORDER BY total_paid DESC
    LIMIT 20
""").df()

spend_dist_df

,cancer_type_name,category,spend_bucket_name,patient_count,total_paid,total_claims
0,Benign Neoplasms,Benign,Medium Cost ($25k-$100k),54,2.152609e+06,12184.0
1,Benign Neoplasms,Benign,Low Cost ($5k-$25k),148,1.862931e+06,21623.0
2,Uncertain Behavior,Uncertain,Low Cost ($5k-$25k),36,4.569612e+05,4593.0
3,Melanoma & Skin,Malignant,Medium Cost ($25k-$100k),11,4.419903e+05,2357.0
4,Benign Neoplasms,Benign,High Cost (>$100k),3,4.167581e+05,602.0
5,Melanoma & Skin,Malignant,Low Cost ($5k-$25k),26,3.539794e+05,3135.0
6,Uncertain Behavior,Uncertain,Medium Cost ($25k-$100k),10,3.330003e+05,1919.0
7,Male Genital Organs,Malignant,Medium Cost ($25k-$100k),7,2.976955e+05,1586.0
8,Male Genital Organs,Malignant,Low Cost ($5k-$25k),21,2.886531e+05,2627.0
9,Other Neoplasm,Other,Medium Cost ($25k-$100k),7,2.796309e+05,1362.0


In [93]:
spend_by_cancer_df = con.sql("""
    SELECT 
        ct.cancer_type_name,
        ct.category,
        COUNT(DISTINCT f.person_id) as patient_count,
        SUM(f.paid_amount) as total_paid,
        SUM(f.claim_count) as total_claims,
        ROUND(SUM(f.paid_amount) / NULLIF(COUNT(DISTINCT f.person_id), 0), 2) as avg_paid_per_patient
    FROM main.fact_oncology_cost f
    JOIN main.dim_cancer_type ct ON f.cancer_type_key = ct.cancer_type_key
    GROUP BY 1, 2
    ORDER BY total_paid DESC
""").df()

spend_by_cancer_df

,cancer_type_name,category,patient_count,total_paid,total_claims,avg_paid_per_patient
0,Benign Neoplasms,Benign,241,4.541918e+06,36533.0,18846.13
1,Uncertain Behavior,Uncertain,64,8.435100e+05,7688.0,13179.84
2,Melanoma & Skin,Malignant,47,8.239682e+05,6001.0,17531.24
3,Male Genital Organs,Malignant,33,6.025821e+05,4575.0,18260.06
4,Other Neoplasm,Other,20,4.231765e+05,3087.0,21158.83
5,Breast,Malignant,14,3.090559e+05,2559.0,22075.42
6,Hematologic,Malignant,9,2.314145e+05,1873.0,25712.72
7,Urinary Tract,Malignant,11,2.029324e+05,1607.0,18448.40
8,In Situ Neoplasms,In Situ,5,1.350080e+05,923.0,27001.61
9,Respiratory & Intrathoracic,Malignant,4,1.189264e+05,545.0,29731.59


## High Cost Patients Analysis

In [94]:
high_cost_df = con.sql("""
    SELECT 
        p.person_id,
        p.primary_cancer_type,
        ct.category,
        sb.spend_bucket_name,
        SUM(f.paid_amount) as total_paid,
        SUM(f.claim_count) as total_claims,
        p.all_cancer_codes
    FROM main.fact_oncology_cost f
    JOIN main.dim_patient p ON f.person_id = p.person_id
    JOIN main.dim_cancer_type ct ON f.cancer_type_key = ct.cancer_type_key
    JOIN main.dim_spend_bucket sb ON f.spend_bucket_key = sb.spend_bucket_key
    WHERE sb.spend_bucket_key = 1  -- High Cost (>$100k)
    GROUP BY 1, 2, 3, 4, 7
    ORDER BY total_paid DESC
    LIMIT 20
""").df()

high_cost_df

,person_id,primary_cancer_type,category,spend_bucket_name,total_paid,total_claims,all_cancer_codes
0,10124,Benign Neoplasms,Benign,High Cost (>$100k),166505.745377,105.0,"[C7A1, D126, D271]"
1,12284,Benign Neoplasms,Benign,High Cost (>$100k),125344.078149,153.0,"[C44612, C61, D0339, D125, D171, D2239, D2261,..."
2,13289,Benign Neoplasms,Benign,High Cost (>$100k),124908.296561,344.0,"[C3492, D235]"
3,11030,Secondary Neuroendocrine,Malignant,High Cost (>$100k),116259.028908,210.0,"[C801, D481]"


## Care Setting Mix by Cancer Category

In [95]:
care_mix_df = con.sql("""
    SELECT 
        ct.category,
        SUM(CASE WHEN cs.care_setting_code = 'inpatient' THEN f.paid_amount ELSE 0 END) as inpatient,
        SUM(CASE WHEN cs.care_setting_code = 'outpatient' THEN f.paid_amount ELSE 0 END) as outpatient,
        SUM(CASE WHEN cs.care_setting_code = 'office-based' THEN f.paid_amount ELSE 0 END) as office_based,
        SUM(CASE WHEN cs.care_setting_code = 'ancillary' THEN f.paid_amount ELSE 0 END) as ancillary,
        SUM(CASE WHEN cs.care_setting_code = 'other' THEN f.paid_amount ELSE 0 END) as other,
        SUM(f.paid_amount) as total
    FROM main.fact_oncology_cost f
    JOIN main.dim_cancer_type ct ON f.cancer_type_key = ct.cancer_type_key
    JOIN main.dim_care_setting cs ON f.care_setting_key = cs.care_setting_key
    GROUP BY 1
    ORDER BY total DESC
""").df()

care_mix_df

,category,inpatient,outpatient,office_based,ancillary,other,total
0,Benign,1.492301e+06,1.286396e+06,1.120101e+06,638083.118722,5036.389968,4.541918e+06
1,Malignant,9.381525e+05,7.860580e+05,6.253130e+05,403710.819159,701.990000,2.753936e+06
2,Uncertain,2.439349e+05,2.492698e+05,2.364091e+05,111587.380170,2308.800016,8.435100e+05
3,Other,2.071155e+05,8.922028e+04,7.549983e+04,51122.539872,218.420000,4.231765e+05
4,In Situ,8.279080e+03,4.480692e+04,7.004100e+04,11881.040031,0.000000,1.350080e+05
5,Unspecified,4.671851e+04,1.893812e+04,1.313889e+04,5993.430020,433.510000,8.522246e+04


## Summary Statistics

In [96]:
summary_df = con.sql("""
    SELECT
        COUNT(DISTINCT p.person_id) as total_patients,
        SUM(f.paid_amount) as total_paid,
        SUM(f.allowed_amount) as total_allowed,
        SUM(f.claim_count) as total_claims,
        ROUND(SUM(f.paid_amount) / NULLIF(COUNT(DISTINCT p.person_id), 0), 2) as avg_paid_per_patient,
        ROUND(SUM(f.paid_amount) / NULLIF(SUM(f.claim_count), 0), 2) as avg_paid_per_claim,
        ROUND(SUM(f.claim_count) * 1.0 / NULLIF(COUNT(DISTINCT p.person_id), 0), 2) as avg_claims_per_patient
    FROM main.fact_oncology_cost f
    JOIN main.dim_patient p ON f.person_id = p.person_id
""").df()

summary_df

,total_patients,total_paid,total_allowed,total_claims,avg_paid_per_patient,avg_paid_per_claim,avg_claims_per_patient
0,475,8.782771e+06,5.499453e+06,68842.0,18490.04,127.58,144.93


## Patient Detail

In [97]:
patient_detail_df = con.sql("""
    SELECT 
        p.person_id,
        p.cohort_name,
        p.primary_cancer_type,
        ct.category as cancer_category,
        p.cancer_claim_count,
        p.most_recent_diagnosis_date,
        sb.spend_bucket_name,
        cs.care_setting_name,
        cs.care_setting_detail_name,
        SUM(f.paid_amount) as total_paid,
        SUM(f.claim_count) as total_claims,
        p.all_cancer_codes,
        p.all_cancer_descriptions
    FROM main.dim_patient p
    LEFT JOIN main.fact_oncology_cost f ON p.person_id = f.person_id
    LEFT JOIN main.dim_cancer_type ct ON f.cancer_type_key = ct.cancer_type_key
    LEFT JOIN main.dim_care_setting cs ON f.care_setting_key = cs.care_setting_key
    LEFT JOIN main.dim_spend_bucket sb ON f.spend_bucket_key = sb.spend_bucket_key
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13
    ORDER BY total_paid DESC NULLS LAST
""").df()

patient_detail_df

,person_id,cohort_name,primary_cancer_type,cancer_category,cancer_claim_count,most_recent_diagnosis_date,spend_bucket_name,care_setting_name,care_setting_detail_name,total_paid,total_claims,all_cancer_codes,all_cancer_descriptions
0,10124,Active Oncology,Benign Neoplasms,Benign,3,2017-08-04,High Cost (>$100k),Inpatient,acute inpatient,159294.595344,19.0,"[C7A1, D126, D271]",[Malignant poorly differentiated neuroendocrin...
1,13289,Active Oncology,Benign Neoplasms,Benign,2,2018-03-14,High Cost (>$100k),Inpatient,acute inpatient,105751.616406,78.0,"[C3492, D235]",[Malignant neoplasm of unspecified part of lef...
2,12284,Active Oncology,Benign Neoplasms,Benign,11,2017-12-04,High Cost (>$100k),Outpatient,pharmacy,99119.238144,5.0,"[C44612, C61, D0339, D125, D171, D2239, D2261,...",[Basal cell carcinoma of skin of right upper l...
3,11030,Active Oncology,Secondary Neuroendocrine,Malignant,2,2017-11-23,High Cost (>$100k),Inpatient,skilled nursing,88039.948592,41.0,"[C801, D481]","[Malignant (primary) neoplasm, unspecified, Ne..."
4,11524,Active Oncology,Respiratory & Intrathoracic,Malignant,61,2018-07-16,Medium Cost ($25k-$100k),Inpatient,acute inpatient,68322.322472,54.0,"[C039, C153, C159, C3411, C3412, C3431, C3432,...","[Malignant neoplasm of gum, unspecified, Malig..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5586,11445,Active Oncology,Uncertain Behavior,Uncertain,3,2018-11-26,Low Cost ($5k-$25k),Ancillary,durable medical equipment,0.000000,1.0,"[C44311, D485, D494]","[Basal cell carcinoma of skin of nose, Neoplas..."
5587,1070,Active Oncology,Melanoma & Skin,Malignant,4,2016-08-07,Low Cost ($5k-$25k),Outpatient,observation,0.000000,1.0,"[C44319, D225, D485]",[Basal cell carcinoma of skin of other parts o...
5588,10145,Active Oncology,Urinary Tract,Malignant,13,2018-03-13,Low Cost ($5k-$25k),Outpatient,pharmacy,0.000000,2.0,"[C3490, C44519, C642, C649, C7951, C801, C8333...",[Malignant neoplasm of unspecified part of uns...
5589,13208,Active Oncology,Melanoma & Skin,Malignant,13,2018-08-01,Medium Cost ($25k-$100k),Inpatient,pharmacy,0.000000,1.0,"[C069, C44310, C44519, C4490, C50912, C649, D0...","[Malignant neoplasm of mouth, unspecified, Bas..."


In [98]:
con.close()